In [1]:
import os
import re
import json
import faiss
import warnings
import huggingface_hub

from openai import OpenAI

from langchain.schema import Document
from langchain_community.vectorstores.faiss import FAISS

from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_upstage import UpstageEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker


os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore", category=FutureWarning)

from dotenv import load_dotenv
load_dotenv("../keys.env")

upstage_api_key = os.getenv("UPSTAGE_API_KEY")
os.environ['UPSTAGE_API_KEY'] = upstage_api_key

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

hf_token = os.getenv("HF_TOKEN")
huggingface_hub.login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/pervinco/.cache/huggingface/token
Login successful


In [2]:
client = OpenAI()
model = "gpt-4o"

# client = OpenAI(
#     api_key=upstage_api_key,
#     base_url="https://api.upstage.ai/v1/solar"
# )
# model = "solar-pro"

In [6]:
def clean_json_response(response):
    # 코드 블록(예: ```json, ```) 제거
    cleaned_response = re.sub(r'```(?:json)?', '', response).strip()
    
    return cleaned_response

def query_expansion(query, model:str, client:OpenAI):
    content = (
        " 당신은 한국어 질의를 이해하여 정확한 문서들을 검색하도록 질의를 개선하는 전문가입니다. "
        " 사용자의 질문 의도를 명확히 파악하고, 그 의도를 유지하면서 검색 시스템이 관련 문서를 더 잘 찾을 수 있도록 질의를 더 명확하게 만드세요. "
        " 생성된 질의는 반드시 사용자의 본래 의도를 반영해야 하며, 새로운 정보나 본래 질의에 포함되지 않은 세부 정보(예: 국가, 인물, 특정 사건 등)를 임의로 추가하지 마세요. "
         "오직 사용자가 제공한 정보만을 바탕으로 질의를 확장하고, 검색에 필수적인 정보만 포함하도록 하세요. "
        " 또한, 확장된 질의는 자연스러운 문장 형태로 제공되어야 합니다. "
        " 반환하는 형식은 반드시 JSON 포맷이어야 하며, 모든 문자열은 쌍따옴표로 감싸야 합니다. "
        '{ "query": "확장된 자연스러운 질의" }.'
    )


    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role" : "system", "content" : content},
            {"role" : "user", "content" : query}
        ],
    )
    
    response = completion.choices[0].message.content
    response = clean_json_response(response)
    
    try:
        json_response = json.loads(response)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON response", "response": response}
    
    return json_response

In [7]:
def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

def load_query(query_path, model, client, output_path):
    queries = load_jsonl(query_path)
    expanded_queries = []

    for query_data in queries:
        eval_id = query_data['eval_id']
        msg_history = query_data['msg']
        
        conversation_history = " ".join([msg['content'] for msg in msg_history if msg['role'] == 'user'])
        
        expanded_query_result = query_expansion(conversation_history, model, client)
        print(eval_id)
        print(conversation_history)
        print(expanded_query_result['query'], '\n')
        
        expanded_queries.append({
            "eval_id": eval_id,
            "original_conversation": conversation_history,
            "expanded_query": expanded_query_result.get('query', 'Expansion failed')
        })
    
    # Save expanded queries to a jsonl file
    with open(output_path, 'w', encoding='utf-8') as f:
        for query in expanded_queries:
            json.dump(query, f, ensure_ascii=False)
            f.write('\n')

In [8]:
load_query("../dataset/eval.jsonl", model, client, "../dataset/expanded_query.jsonl")

78
나무의 분류에 대해 조사해 보기 위한 방법은?
나무의 분류를 조사하는 방법 

213
각 나라에서의 공교육 지출 현황에 대해 알려줘.
각 나라의 공교육 지출 현황 

107
기억 상실증 걸리면 너무 무섭겠다. 어떤 원인 때문에 발생하는지 궁금해.
기억 상실증의 발생 원인 

81
통학 버스의 가치에 대해 말해줘.
통학 버스의 가치와 중요성 

280
Dmitri Ivanovsky가 누구야?
Dmitri Ivanovsky라는 인물에 대한 정보 

10
피임을 하기 위한 방법중 약으로 처리하는 방법은 쓸만한가?
피임을 위한 방법 중 약을 사용하는 것이 효과적인지 평가 

100
헬륨이 다른 원소들과 반응을 잘 안하는 이유는?
헬륨이 다른 원소들과 반응을 잘 하지 않는 이유 

279
문맹 비율이 사회 발전에 미치는 영향은?
문맹 비율이 사회 발전에 미치는 영향 

42
이란 콘트라 사건이 뭐야 이 사건이 미국 정치에 미친 영향은?
이란 콘트라 사건의 내용과 이 사건이 미국 정치에 미친 영향 

308
자기장이 얼마나 센지 표현하는 방식은?
자기장의 세기를 표현하는 방식 

205
피를 맑게 하고 몸 속의 노폐물을 없애는 역할을 하는 기관은?
피를 맑게 하고 몸 속의 노폐물을 제거하는 인체 기관 

289
글리코겐의 분해는 인체에서 왜 필요한가?
인체에서 글리코겐 분해의 필요성 

268
빗방울이 점점 커지게 되는 요인은?
빗방울이 점점 커지게 되는 요인 

18
기체의 부피나 형태가 왜 일정하지 않을까?
기체의 부피와 형태가 일정하지 않은 이유 

9
식물이 빛을 에너지로 변환하는 과정에 대해 설명해줘.
식물이 빛을 에너지로 변환하는 광합성 과정에 대한 설명 

101
직류와 교류 전류의 차이에 대해 알려줘.
직류 전류와 교류 전류의 차이 

236
기름과 물이 섞여 있을 수 있나?
기름과 물의 혼합 가능성에 대한 설명 

59
인간이 2세를 생산할 때 DNA의 결합 과정에 대히 설명해줘.
인간이 2세를 생산할 때 DNA의 결합 과정 설명 

25
금성에서 

KeyError: 'query'